In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import os, pathlib, glob, random
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix
import scipy
from scipy import io

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
batch_size = 32
output_nodes = 2
learning_rate = 0.003

In [4]:
train_data_path = r"/kaggle/input/jpd-df2-lfcc-t2/LFCC/train"
# test_data_path =r"/kaggle/input/t1-lfcc-mladdc/LFCC_T1/test"
validation_data_path= r"/kaggle/input/jpd-df2-lfcc-t2/LFCC/val"

In [5]:
class PtDataset(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
        self.files = []
        for c in self.classes:
            c_dir = os.path.join(directory, c)
            c_files = [(os.path.join(c_dir, f), self.class_to_idx[c]) for f in os.listdir(c_dir)]
            self.files.extend(c_files)
        random.shuffle(self.files)
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        filepath, label = self.files[idx]
        try:
            mat_vals = scipy.io.loadmat(filepath)
            data = mat_vals['final']
            data = data.T
            max_len=800
            if (max_len > data.shape[0]):
                pad_width = max_len - data.shape[0]
                data = np.pad(data, pad_width=((0, pad_width),(0,0)), mode='constant')
            else:
                data = data[:max_len, :]
        except Exception as e:
            print(f"Error loading file {filepath}: {str(e)}")
            return None
        return data, label

In [6]:
train_dataset = PtDataset(train_data_path)
# test_dataset = PtDataset(test_data_path)
val_dataset = PtDataset(validation_data_path)

In [7]:
class PtDataLoader(DataLoader):
    def __init__(self, directory, batch_size, shuffle=True):
        dataset = PtDataset(directory)
        super().__init__(dataset, batch_size=batch_size, shuffle=shuffle)

In [8]:
train_dataloader = PtDataLoader(directory=train_data_path, batch_size=batch_size)
# test_dataloader = PtDataLoader(directory=test_data_path, batch_size=batch_size)
val_dataloader = PtDataLoader(directory=validation_data_path, batch_size=batch_size)

In [9]:
train_count = len(train_dataset) 
# test_count = len(test_dataset)
val_count = len(val_dataset)

In [10]:
print(train_count)
# print(test_count)
print(val_count)

137198
29402


In [11]:
import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F

In [12]:
# Define the parameters
input_size = 20
hidden_size = 256
num_layers = 2
num_classes = 2
# drop_amount = 0.25  # You can choose an appropriate dropout rate

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

In [14]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import os, pathlib, glob, random
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import confusion_matrix
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
from transformers.models.whisper.modeling_whisper import WhisperModel, WhisperEncoder
from transformers.models.whisper.configuration_whisper import WhisperConfig
from typing import Optional, Tuple, Union
import torch
import librosa 
import matplotlib.pyplot as plt
import numpy as np
import os, glob, pickle
import scipy.io as sio
from tqdm import tqdm
import multiprocessing as mp 
import torch.optim as optim

In [15]:
# BiLSTM

In [16]:
drop_amount = 0.255

class BiLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(p=drop_amount)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device=x.device, dtype=torch.double)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device=x.device, dtype=torch.double)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out)
        # Extract the output of the last time step from both directions
        last_hidden_state = torch.cat((out[:, -1, :self.hidden_size], out[:, 0, self.hidden_size:]), dim=1)
        output = self.fc(last_hidden_state)
        return output

In [17]:
batch_size = 32
output_nodes = 2
learning_rate = 0.003

In [18]:
model = BiLSTMClassifier(input_size, hidden_size, num_layers, num_classes)
model.to(device, dtype=torch.double)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)

BiLSTMClassifier(
  (lstm): LSTM(20, 256, num_layers=2, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.255, inplace=False)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)


In [19]:
print(model)

BiLSTMClassifier(
  (lstm): LSTM(20, 256, num_layers=2, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.255, inplace=False)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)


In [20]:
from tqdm import tqdm
import torch
from torch.autograd import Variable

# Model training and testing
n_total_steps = len(train_dataloader)
train_accuracy_list = []
train_loss_list = []
val_accuracy_list = []
max_acc = 0
num_epochs = 15
pred_labels = []
act_labels = []

for epoch in range(num_epochs):
    
    # Training phase
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    print(f"\nEpoch {epoch + 1}/{num_epochs} - Training...")
    
    # Use tqdm for progress bar in training loop
    for batch_idx, (images, labels) in enumerate(tqdm(train_dataloader, desc="Training Batches")):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
    
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    
    train_accuracy_list.append(train_accuracy)
    train_loss_list.append(train_loss)

    # Validation phase
    model.eval()
    val_accuracy = 0.0
    pred = []
    lab = []
    
    print(f"Epoch {epoch + 1}/{num_epochs} - Validation...")
    
    # Use tqdm for progress bar in validation loop
    for i, (images, labels) in enumerate(tqdm(val_dataloader, desc="Validation Batches")):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        val_accuracy += int(torch.sum(prediction == labels.data))
        
        pred.extend(prediction.tolist())
        lab.extend(labels.tolist())
    
    val_accuracy = val_accuracy / val_count
    val_accuracy_list.append(val_accuracy)
    
    # Save best model
    if max_acc < val_accuracy:
        max_acc = val_accuracy
        pred_labels = pred
        actual_labels = lab
        torch.save(model, "best_accuracy_model_BiLSTM.pth")
    
    print(f'Epoch: {epoch + 1}/{num_epochs}   '
          f'Train Loss: {train_loss:.4f}   '
          f'Train Accuracy: {train_accuracy:.4f}   '
          f'Validation Accuracy: {val_accuracy:.4f}')

print('\nMaximum Validation Accuracy:', max_acc)
print('Finished Training and Validation')



Epoch 1/15 - Training...


Training Batches: 100%|██████████| 4288/4288 [31:19<00:00,  2.28it/s]


Epoch 1/15 - Validation...


Validation Batches: 100%|██████████| 919/919 [04:07<00:00,  3.71it/s]


Epoch: 1/15   Train Loss: 0.3537   Train Accuracy: 0.8347   Validation Accuracy: 0.9493

Epoch 2/15 - Training...


...

Epoch: 14/15   Train Loss: 0.0571   Train Accuracy: 0.9800   Validation Accuracy: 0.9824

Epoch 15/15 - Training...


Training Batches: 100%|██████████| 4288/4288 [25:12<00:00,  2.84it/s]


Epoch 15/15 - Validation...


Validation Batches: 100%|██████████| 919/919 [02:58<00:00,  5.15it/s]

Epoch: 15/15   Train Loss: 0.0576   Train Accuracy: 0.9803   Validation Accuracy: 0.9817

Maximum Validation Accuracy: 0.982416162165839
Finished Training and Validation


In [21]:
# # Load the best model
# best_model = torch.load("best_accuracy_model_BiLSTM.pth")

# # Put the best_model in evaluation mode
# best_model.eval()

# # Initialize variables to store results
# testing_accuracy = 0.0
# pred_labels = []
# act_labels = []

# # Pass validation data through the best model
# for i, (images, labels) in enumerate(test_dataloader):
#     if torch.cuda.is_available():
#         images = Variable(images.cuda())
#         labels = Variable(labels.cuda())
    
#     outputs = best_model(images)
#     _, prediction = torch.max(outputs.data, 1)
    
#     testing_accuracy += int(torch.sum(prediction == labels.data))
    
#     pred_labels.extend(prediction.tolist())
#     act_labels.extend(labels.tolist())

# # Calculate validation accuracy
# testing_accuracy = testing_accuracy / len(test_dataloader.dataset)

# # Print the validation accuracy
# print("testing Accuracy:", testing_accuracy)

In [22]:
# # Calculate the confusion matrix
# import seaborn as sns
# conf_mat = confusion_matrix(act_labels, pred_labels)
# # Plot confusion matrix heat map
# sns.heatmap(conf_mat, cmap="flare",annot=True, fmt = "g", 
#             cbar_kws={"label":"color bar"},
#             xticklabels=train_dataset.classes,
#             yticklabels=train_dataset.classes)
# plt.xlabel("Predicted")
# plt.ylabel("Actual")
# plt.title("Confusion Matrix")
# plt.savefig("ConfusionMatrix_BiLSTM.png")
# plt.show()
# from sklearn.metrics import f1_score
# f1_score = f1_score(pred_labels, act_labels, average='macro')
# print('F1 Score : ', f1_score)


In [23]:
# import numpy as np
# import sklearn.metrics

# """
# Python compute equal error rate (eer)
# ONLY tested on binary classification

# :param label: ground-truth label, should be a 1-d list or np.array, each element represents the ground-truth label of one sample
# :param pred: model prediction, should be a 1-d list or np.array, each element represents the model prediction of one sample
# :param positive_label: the class that is viewed as positive class when computing EER
# :return: equal error rate (EER)
# """
# def compute_eer(label, pred):
#     # all fpr, tpr, fnr, fnr, threshold are lists (in the format of np.array)
#     fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred)
#     fnr = 1 - tpr

#     # the threshold of fnr == fpr
#     eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]

#     # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
#     eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
#     eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]

#     # return the mean of eer from fpr and from fnr
#     eer = (eer_1 + eer_2) / 2
#     return eer

# eer = compute_eer(act_labels, pred_labels)
# print('The equal error rate is {:.3f}'.format(eer))